In [27]:
import os
from glob import glob
import shutil
import subprocess
from subprocess import run

from rdkit import Chem

In [28]:
nodes = [i.split('_')[1].split('.sdf')[0] for i in os.listdir('data') if 'conformers' in i]

for node in nodes:
    print(f'Now on node {node}')
    TARGET_DIR = f'data/sdf/{node}/'
    INPUT_SMILES=f'data/fragments_smi/frag_{node}_unique.txt'
    OUTPUT_TEMPLATE='hMOF'
    OUT_DIR=f'data/fragments_all/{node}/'
    CORES='32'
    
    # generate sdf of molecular fragments
    os.mkdir(TARGET_DIR)
    run(['python','-W','ignore','rdkit_conf_parallel.py',INPUT_SMILES,OUTPUT_TEMPLATE,'--cores',CORES],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    for sdf in glob('*.sdf'):
        shutil.move(sdf,TARGET_DIR) 
    
    # generate sdf for frags and linkers
    os.mkdir(OUT_DIR)
    run(['python','-W','ignore','prepare_dataset_parallel.py','--table',INPUT_SMILES,'--sdf-dir',TARGET_DIR,'--out-dir',OUT_DIR,'--template',OUTPUT_TEMPLATE,'--cores',CORES])

    # filter and merge fragments
    run(['python','-W','ignore','filter_and_merge.py','--in-dir',OUT_DIR,'--out-dir',OUT_DIR,'--template',OUTPUT_TEMPLATE,'--number-of-files',CORES])

Now on node [V]


100%|██████████| 64/64 [00:00<00:00, 30059.96it/s]


Now on node [Cu][Cu]


100%|██████████| 127/127 [00:00<00:00, 51922.86it/s]


Now on node [Zn][O]([Zn])([Zn])[Zn]


100%|██████████| 211/211 [00:00<00:00, 55492.74it/s]


Now on node [Zn][Zn]


100%|██████████| 96/96 [00:00<00:00, 51332.63it/s]
